In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import load_pkl, GRAY_DASH, save_latex, FLAVOR_NAMES, ARCH_NAMES, texttt, footnotesize

data: DiffResult = load_pkl("config")

ISSUES = {
    IssueEnum.ADD: "$+$",
    IssueEnum.REMOVE: "$-$",
    IssueEnum.CHANGE: r"$\Delta$",
    IssueEnum.NEW: "#",
}

KINDS = {
    DepKind.STRUCT: "Struct",
    DepKind.CONFIG: "Config",
    DepKind.FUNC: "Func",
    DepKind.TRACEPOINT: r"\shortstack{Tracept\\($\Delta=0$)}",
    DepKind.SYSCALL: r"Syscall",
}


def format_val(v):
    if v > 1000:
        return f"{v/1000:.1f}k"
    return v


results = {}
for i, ((g, p), d) in enumerate(data.items()):
    if i == 0:
        result = {}
        for (kind, s), v in d.items():
            if s == IssueEnum.OLD:
                result[(KINDS[kind], ISSUES[IssueEnum.NEW])] = format_val(v)
            elif (
                s in (IssueEnum.ADD, IssueEnum.REMOVE, IssueEnum.CHANGE)
                and kind != DepKind.CONFIG
            ):
                result[(KINDS[kind], ISSUES[s])] = 0
        result[("Register", ISSUES[IssueEnum.CHANGE])] = 0
        results[(r"\multicolumn{1}{c|}{def-}", r"\multicolumn{1}{c|}{ault}")] = result

    result = {}
    for (kind, s), v in d.items():
        if kind == DepKind.CONFIG and s != IssueEnum.NEW:
            continue
        kind = KINDS[kind]
        s = ISSUES.get(s)
        if not s:
            continue
        result[(kind, s)] = format_val(v)
    result[("Register", ISSUES[IssueEnum.CHANGE])] = 1 if g == VersionGroup.ARCH else 0
    name = g.to_str(p.v2)
    if g == VersionGroup.ARCH:
        g = "Architecture"
        name = texttt(footnotesize(ARCH_NAMES[name]))
    elif g == VersionGroup.FLAVOR:
        name = FLAVOR_NAMES[name]
    results[(g, name)] = result


df = pd.DataFrame(results)
df = df.loc[(df != 0).any(axis=1)]


df_latex = df.map(lambda x: GRAY_DASH if x == 0 else x)
latex = df_latex.to_latex(
    multicolumn_format="c|", column_format=r"p{7ex}p{1.5ex}|r|rrrr|rrrr", multirow=True
)

save_latex(latex, "cfg", rotate=False)
df

[ utils_pickle.py:18 ] INFO: Loding config from /Users/szhong/Code/DepSurf/output/config.pkl
[  utils_latex.py:77 ] INFO: Saved cfg to /Users/szhong/Code/DepSurf/paper/tabs/cfg.tex


\multicolumn{1}{c|}{def-}                  Architecture                                                                                         Flavor                      
                                            \multicolumn{1}{c|}{ault} \texttt{\footnotesize{arm64}} \texttt{\footnotesize{arm32}} \texttt{\footnotesize{ppc}} \texttt{\footnotesize{riscv}}    AWS  Azure    GCP low-lat
Config                             #                             8.8k                          9.6k                          9.6k                        8.1k                          7.6k   6.4k   5.3k   8.6k    8.8k
Func                               #                            48.0k                         49.4k                         48.8k                       42.8k                         36.6k  46.5k  45.5k  48.1k   48.0k
                                   $+$                              0                          9.2k                         12.6k                        5.4k                          2.1k    328    992    450      57
                                   $-$                              0                          7.9k                         11.8k                       10.6k                         13.5k   1.8k   3.5k    319      41
                                   $\Delta$                         0                           120                           106                         137                           101      2     10      1       0
Struct                             #                             8.4k                          9.1k                          8.6k                        7.4k                          6.6k   8.0k   7.8k   8.4k    8.4k
                                   $+$                              0                          1.7k                          2.0k                         570                           157     83    257     68       4
                                   $-$                              0                          1.0k                          1.9k                        1.6k                          2.0k    483    833    123       1
                                   $\Delta$                         0                            81                           154                         116                            98     19     28     14       5
\shortstack{Tracept\\($\Delta=0$)} #                              752                           685                           690                         648                           625    747    739    752     752
                                   $+$                              0                            45                            70                          25                             0      4     26      0       0
                                   $-$                              0                           112                           132                         129                           127      9     39      0       0
Syscall                            #                              333                           291                           378                         347                           280    333    333    333     333
                                   $+$                              0                             2                            74                          23                             2      0      0      0       0
                                   $-$                              0                            44                            29                           9                            55      0      0      0       0
Register                           $\Delta$                         0                             1                             1                           1                             1      0      0      0       0